# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys_BOOOOOTH import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities_booooth.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,invercargill,-46.4000,168.3500,51.03,76,79,18.99,NZ,1700006949
1,1,wailua homesteads,22.0669,-159.3780,84.63,71,75,23.02,US,1700007291
2,2,grytviken,-54.2811,-36.5092,37.26,92,81,1.52,GS,1700007292
3,3,waitangi,-43.9535,-176.5597,59.83,88,82,3.00,NZ,1700007293
4,4,ha'il,27.5219,41.6907,62.62,100,0,6.91,SA,1700007293


In [3]:
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     48 non-null     int64  
 1   City        48 non-null     object 
 2   Lat         48 non-null     float64
 3   Lng         48 non-null     float64
 4   Max Temp    48 non-null     float64
 5   Humidity    48 non-null     int64  
 6   Cloudiness  48 non-null     int64  
 7   Wind Speed  48 non-null     float64
 8   Country     48 non-null     object 
 9   Date        48 non-null     int64  
dtypes: float64(4), int64(4), object(2)
memory usage: 3.9+ KB


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
# Configure the map
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "CartoDark",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City"
)

# Display the map plot
map_plot

C:\Users\abooth\AppData\Local\anaconda3\envs\PythonData\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,invercargill,-46.4000,168.3500,51.03,76,79,18.99,NZ,1700006949
1,1,wailua homesteads,22.0669,-159.3780,84.63,71,75,23.02,US,1700007291
2,2,grytviken,-54.2811,-36.5092,37.26,92,81,1.52,GS,1700007292
3,3,waitangi,-43.9535,-176.5597,59.83,88,82,3.00,NZ,1700007293
4,4,ha'il,27.5219,41.6907,62.62,100,0,6.91,SA,1700007293


In [22]:
# Narrow down cities that fit criteria and drop any results with null values
# temp >= 70, <80
# wind < 10 MPH
# cloudiness = 0

# make the max temp >= filter 70 for the real thing
mask = (city_data_df["Max Temp"] >= 50) & (city_data_df["Max Temp"] < 80) & (city_data_df["Wind Speed"] < 10) & (city_data_df["Cloudiness"] == 0)
df_sub = city_data_df.loc[mask]

# Drop any rows with null values
df_sub = df_sub.dropna(how="any")

# Display sample data
df_sub.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,ha'il,27.5219,41.6907,62.62,100,0,6.91,SA,1700007293
15,15,songjiang,31.0360,121.2146,53.56,48,0,2.64,CN,1700007299
30,30,fort bragg,35.1390,-79.0060,51.80,60,0,0.00,US,1700007311
38,38,remire-montjoly,4.9167,-52.2667,78.84,94,0,2.30,GF,1700007316


### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = df_sub.loc[:, ["City", "Country", "Lat", "Lng", "Humidity", "Max Temp", "Cloudiness"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Max Temp,Cloudiness,Hotel Name
4,ha'il,SA,27.5219,41.6907,100,62.62,0,
15,songjiang,CN,31.0360,121.2146,48,53.56,0,
30,fort bragg,US,35.1390,-79.0060,60,51.80,0,
38,remire-montjoly,GF,4.9167,-52.2667,94,78.84,0,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [24]:
# Set parameters to search for a hotel
categories = "accommodation.hotel"
radius = 10000
limit = 20
params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row.Lat
    longitude = row.Lng
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ha'il - nearest hotel: فندق كلاسيك
songjiang - nearest hotel: 上海松江开元名都大酒店
fort bragg - nearest hotel: Airborne Inn Lodging
remire-montjoly - nearest hotel: Complexe Belova


,City,Country,Lat,Lng,Humidity,Max Temp,Cloudiness,Hotel Name
4,ha'il,SA,27.5219,41.6907,100,62.62,0,فندق كلاسيك
15,songjiang,CN,31.0360,121.2146,48,53.56,0,上海松江开元名都大酒店
30,fort bragg,US,35.1390,-79.0060,60,51.80,0,Airborne Inn Lodging
38,remire-montjoly,GF,4.9167,-52.2667,94,78.84,0,Complexe Belova


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [26]:
# Configure the map
map_plot2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "CartoDark",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    hover_cols=["Lat", "Lng", "City", "Country", "Hotel Name", "Max Temp"]
)

# Display the map plot
map_plot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name,Max Temp)